# Query models and their pre-trained checkpoints on different datasets

In [6]:
import pandas as pd
import tensorflow as tf

pd.options.display.max_colwidth = None

In [2]:
# Load master table from Cloud.
with tf.io.gfile.GFile('gs://vit_models/augreg/index.csv') as f:
    df = pd.read_csv(f)

2022-12-05 03:12:13.583956: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


In [3]:
df.name.unique()

array(['Ti/16', 'S/32', 'B/16', 'L/16', 'R50+L/32', 'R26+S/32', 'S/16',
       'B/32', 'R+Ti/16', 'B/8'], dtype=object)

In [12]:
len(df.filename.unique())
# df.filename.unique()

759

In [16]:
df.adapt_ds.unique()

array(['imagenet2012', 'cifar100', 'resisc45', 'oxford_iiit_pet', 'kitti'],
      dtype=object)

In [4]:
# Let's first select the "best checkpoint" for every model. We show in the
# paper (section 4.5) that one can get a good performance by simply choosing the
# best model by final pre-train validation accuracy ("final-val" column).
# Pre-training with imagenet21k 300 epochs (ds=="i21k") gives the best
# performance in almost all cases (figure 6, table 5).
best_filenames = set(
    df.query('ds=="i21k"')
    .groupby('name')
    .apply(lambda df: df.sort_values('final_val').iloc[-1])
    .filename
)

# Select all finetunes from these models.
best_df = df.loc[df.filename.apply(lambda filename: filename in best_filenames)]

In [29]:
# Note that this dataframe contains the models from the "i21k_300" column of
# table 3:
best_df.query('adapt_ds=="imagenet2012"').groupby(['name', 'adapt_resolution']).apply(
    lambda df: df.sort_values('adapt_final_test').iloc[-1]
)[[
   # Columns from upstream
   'name', 'params', 'ds', 'filename',
   # Columns from downstream
   'adapt_resolution', 'infer_samples_per_sec','adapt_ds', 'adapt_final_test', 'adapt_filename',
]].sort_values('params')

name       params    ds  \
name     adapt_resolution                                
Ti/16    224                  Ti/16    5720000.0  i21k   
         384                  Ti/16    5790000.0  i21k   
R+Ti/16  224                R+Ti/16    6340000.0  i21k   
         384                R+Ti/16    6360000.0  i21k   
S/16     224                   S/16   22050000.0  i21k   
         384                   S/16   22200000.0  i21k   
S/32     224                   S/32   22880000.0  i21k   
         384                   S/32   22920000.0  i21k   
R26+S/32 224               R26+S/32   36430000.0  i21k   
         384               R26+S/32   36470000.0  i21k   
B/16     224                   B/16   86570000.0  i21k   
         384                   B/16   86860000.0  i21k   
B/32     224                   B/32   88220000.0  i21k   
         384                   B/32   88300000.0  i21k   
R50+L/32 224               R50+L/32  110950000.0  i21k   
         384               R50+L/32  111020000.0  i21k   
L/16     224                   L/16  304330000.0  i21k   
         384                   L/16  304720000.0  i21k   
B/8      224                    B/8          NaN  i21k   

                                                                                filename  \
name     adapt_resolution                                                                  
Ti/16    224                    Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0   
         384                    Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0   
R+Ti/16  224                  R_Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0   
         384                  R_Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0   
S/16     224                   S_16-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0   
         384                   S_16-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0   
S/32     224                      S_32-i21k-300ep-lr_0.001-aug_none-wd_0.1-do_0.0-sd_0.0   
         384                      S_32-i21k-300ep-lr_0.001-aug_none-wd_0.1-do_0.0-sd_0.0   
R26+S/32 224                R26_S_32-i21k-300ep-lr_0.001-aug_light1-wd_0.1-do_0.0-sd_0.0   
         384                R26_S_32-i21k-300ep-lr_0.001-aug_light1-wd_0.1-do_0.0-sd_0.0   
B/16     224                   B_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.0-sd_0.0   
         384                   B_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.0-sd_0.0   
B/32     224                    B_32-i21k-300ep-lr_0.001-aug_light1-wd_0.1-do_0.0-sd_0.0   
         384                    B_32-i21k-300ep-lr_0.001-aug_light1-wd_0.1-do_0.0-sd_0.0   
R50+L/32 224               R50_L_32-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.1-sd_0.1   
         384               R50_L_32-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.1-sd_0.1   
L/16     224                   L_16-i21k-300ep-lr_0.001-aug_strong1-wd_0.1-do_0.0-sd_0.0   
         384                   L_16-i21k-300ep-lr_0.001-aug_strong1-wd_0.1-do_0.0-sd_0.0   
B/8      224                    B_8-i21k-300ep-lr_0.001-aug_medium2-wd_0.1-do_0.0-sd_0.0   

                           adapt_resolution  infer_samples_per_sec  \
name     adapt_resolution                                            
Ti/16    224                            224                3097.42   
         384                            384                 609.58   
R+Ti/16  224                            224                9371.00   
         384                            384                2425.77   
S/16     224                            224                1508.35   
         384                            384                 300.12   
S/32     224                            224                8342.46   
         384                            384                2153.94   
R26+S/32 224                            224                1814.25   
         384                            384                 560.40   
B/16     224                           